In [14]:
import tkinter as tk
import numpy as np
import os
import cv2
import face_recognition
from datetime import datetime
from math import sqrt

In [15]:
faces_images=[]
faces_labels=[]

faces_file=os.listdir("Faces")

for person in faces_file:
    try:
        faces=os.listdir(f"Faces/{person}")
        for face in faces:
            try:
                current_face=cv2.imread(f"Faces/{person}/{face}")
                if current_face is not None:
                    faces_images.append(current_face)
                    faces_labels.append(person)
                else:
                    print(f"Faces/{person}/{current_face} is None type")
            except Exception as e:
                print(f"Erreur avec Faces/{person}/{current_face} : {e}")
    except Exception as ex:
        print(f"Erreur avec Faces/{person} : {ex}")

Erreur avec Faces/pairs.txt : [Errno 20] Not a directory: 'Faces/pairs.txt'
Erreur avec Faces/pairs_01.txt : [Errno 20] Not a directory: 'Faces/pairs_01.txt'
Erreur avec Faces/pairs_02.txt : [Errno 20] Not a directory: 'Faces/pairs_02.txt'
Erreur avec Faces/pairs_03.txt : [Errno 20] Not a directory: 'Faces/pairs_03.txt'
Erreur avec Faces/pairs_04.txt : [Errno 20] Not a directory: 'Faces/pairs_04.txt'
Erreur avec Faces/pairs_05.txt : [Errno 20] Not a directory: 'Faces/pairs_05.txt'
Erreur avec Faces/pairs_06.txt : [Errno 20] Not a directory: 'Faces/pairs_06.txt'
Erreur avec Faces/pairs_07.txt : [Errno 20] Not a directory: 'Faces/pairs_07.txt'
Erreur avec Faces/pairs_08.txt : [Errno 20] Not a directory: 'Faces/pairs_08.txt'
Erreur avec Faces/pairs_09.txt : [Errno 20] Not a directory: 'Faces/pairs_09.txt'
Erreur avec Faces/pairs_10.txt : [Errno 20] Not a directory: 'Faces/pairs_10.txt'


In [38]:
def imgProcess(img):
    try:
        img.dtype
    except:
        print("convertion de l'img")
        img=cv2.imread(img)
    gris=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #RGB -> Gray -> dim(255,255,3)=255*255*3 val de couleur -> dim(255,255,1) -> 255*255*1 val de couleur
    resize=cv2.resize(gris,(100,100)) #dim -> dim(100,100,1) -> 100*100*1 = 10000 val de couleur
    flat=resize.flatten() #2D -> 1D vec de (10000,1)
    return flat

processed_faces=[]
for img in faces_images:
    processed_faces.append(imgProcess(img))

In [ ]:
def distEuclid(img1,img2): #p1(a1,a2,a3...,a10000) p2(b1,b2,b3...,b10000) car chaque image a 10000 val de couleur
    img1 = img1.astype(np.int32) #pour avoir des valeur négatives sinon en uint8 c'est de 0 à 255 (pas negatif)
    img2 = img2.astype(np.int32)

    #distance euclidienne entre deux points (en 2D) p1(x1,y1) et p2(x2,y2): d=hypothénuse du triangle rectangle
    #ainsi d²=a²+b² et d=sqrt(a²+b²) avec a=abs(x1-x2) et b=abs(y1-y2) DONC -> d=sqrt((x1-x2)²+(y1-y2)²)
    #en n dim : d=sqrt((a1-b1)²+(a2-b2)²...+(an-bn)²) = sqrt(sum[1;N]((a-b)²))
    return sqrt(sum((a-b)**2 for a,b in zip(img1,img2))) #zip -> pour deux listes

In [34]:
def knn_predict(X_train,y_train, newImg,k=3):
    distances=[]
    for i in range(len(X_train)):
        dist=distEuclid(X_train[i],newImg) #on calcule la distance entre la nouvelle image et toutes les autres images
        distances.append((dist,y_train[i])) #on stock la distance avec le label de chaque image à laquelle on a calculé la distance
    
    distances.sort(key=lambda x: x[0]) #on trie la liste de distances dans l'ordre croissant comme ça les derniers termes sont les plus proche de la nouvelle image
    k_voisins=distances[:k] #on prend les k plus proches voisins

    counts={} #dictionnaire pour compter les labels les plus proches et leur occurences
    for _,label in k_voisins: #dans le tableau c'est [distance,label], ici on veut que le label donc _
        counts[label]=counts.get(label,0)+1 #.get(a,b) = prend la val a si existe sinon prend la val b

    return max(counts,key=counts.get) #compare les valeurs mais renvoie le label du dictionnaire

In [40]:
print(knn_predict(processed_faces,faces_labels,imgProcess("test.jpg")))

convertion de l'img
Aaron_Eckhart


In [ ]:
def get_face_encodings(images):
    encoding_list=[]
    for i,image in enumerate(images):
        imageE=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        face_encoding=face_recognition.face_encodings(imageE)[0]
        encoding_list.append(face_encoding)
    return encoding_list

known_face_encodings=get_face_encodings(faces_images)


RuntimeError: Unsupported image type, must be 8bit gray or RGB image.

In [ ]:
def document_recognised_face(name,filename="records.csv"):
    capture_date=datetime.now().strftime("%Y-%m-d")
    if not os.path.isfile(filename):
        with open(filename,"w") as f:
            f.write("Name,Date,Time")
        
    with open(filename,"r+") as file:
        lines=file.readlines()
        existing_names=[line.split(",")[0] for line in lines]
        if name not in existing_names:
            now=datetime.now()
            current_time=now.strftime("%H:%M:%S")
            file.write(f"\n{name},{capture_date},{current_time}")

In [ ]:
def start_recognition_program():
    video_capture=cv2.VideoCapture(1)
    while True:
        frame=video_capture.read()
        if frame is not None:
            frame=frame[1]
            resized_frame=cv2.resize(frame,(0,0),None,0.25,0.25)
            resized_frame=cv2.cvtColor(resized_frame,cv2.COLOR_BGR2RGB)

            face_locations=face_recognition.face_locations(resized_frame)
            current_face_encodings=face_recognition.face_encodings(resized_frame,face_locations)
            for face_encoding, location in zip(current_face_encodings,face_locations):
                matches=face_recognition.compare_faces(known_face_encodings,face_encoding)
                face_distances=face_recognition.face_distance(known_face_encodings,face_encoding)
                best_match_index=np.argmin(face_distances)
                if matches[best_match_index]:
                    recognized_name=faces_labels[best_match_index].upper()
                    top,right,bottom,left=location
                    top,right,bottom,left=top*4,right*4,bottom*4,left*4
                    cv2.rectangle(frame,(left,top),(right,bottom),(0,255,0),2)
                    cv2.rectangle(frame,(left,bottom-35),(right,bottom),(0.255,0),cv2.FILLED)
                    cv2.putText(frame,recognized_name,(left+6,bottom-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
                    document_recognised_face(recognized_name)
            cv2.imshow("Webcam",frame)
        key=cv2.waitKey(1) & 0xFF
        if key==ord("q"):
            break
    video_capture.release()
    cv2.destroyAllWindows()

In [ ]:
root=tk.Tk()
root.title("Face Recognition Program")

label=tk.Label(root,text="Click the button to start the facial recognition program")
label.pack(pady=10)

start_button=tk.Button(root,text="Start Recognition",command=start_recognition_program)
start_button.pack(pady=10)

def quit_app():
    root.quit()
    cv2.destroyAllWindows()

exit_button=tk.Button(root,text="Close",command=quit_app)
exit_button.pack(pady=10)

root.mainloop()